# Università degli Studi di Milano Bicocca  <br/> Master's Degree in Data Science

## Final Project - Foundation of Computer Science (F9101Q001)

**Gruppo 35**: <br/>
Daniele Rizzo *872359*<br/>
Carlo Salaorni *760080*<br/>
Matteo Tarli *800676*

<a href = "https://github.com/gdv/foundationsCS/blob/master/progetti/2021-students.ipynb" title = 2021-students.ipynb>Assignment Body</a>

Definizione delle librerie di interesse e dei dataset di partenza. <br/>
La libreria `Pandas` verrà sfruttata per la lettura ed elaborazione dei dataframe, mentre la libreria `re` per l'uso dell'espressioni regolari. Verrà infine sfruttata la libreria `time` per registrare il tempo necessario per una run completa del codice.

In [1]:
import time
start = time.time()

In [2]:
import pandas as pd 
import re

In [3]:
times = pd.read_csv('https://github.com/DanieleRizz0/CSProject/raw/main/timesData.csv', thousands = ',')
shangai = pd.read_csv('https://github.com/DanieleRizz0/CSProject/raw/main/shanghaiData.csv')
cwur = pd.read_csv('https://github.com/DanieleRizz0/CSProject/raw/main/cwurData.csv')

In [4]:
times.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
0,1,Harvard University,United States of America,99.7,72.4,98.7,98.8,34.5,96.1,20152.0,8.9,25%,NaN,2011
1,2,California Institute of Technology,United States of America,97.7,54.6,98.0,99.9,83.7,96.0,2243.0,6.9,27%,33 : 67,2011
2,3,Massachusetts Institute of Technology,United States of America,97.8,82.3,91.4,99.9,87.5,95.6,11074.0,9.0,33%,37 : 63,2011
3,4,Stanford University,United States of America,98.3,29.5,98.1,99.2,64.3,94.3,15596.0,7.8,22%,42 : 58,2011
4,5,Princeton University,United States of America,90.9,70.3,95.4,99.9,-,94.2,7929.0,8.4,27%,45 : 55,2011


Si può notare come l'attributo `income` di interesse risulti essere di tipo `object`. Al fine di correggere questa situazione per utilizzi futuri di questa colonna, si effettua una conversione dell'attributo in tipo `numeric`.

In [5]:
times.income.dtype

dtype('O')

Si noti che il record 4 presenta il valore `-` in corrispondenza dell'attributo `income`. Viene quindi utilizzato l'argomento `errors = 'coerce'` della funzione di Pandas `to_numeric` al fine di convertire tale valore anomalo in valore nullo.

In [6]:
times.income = pd.to_numeric(times.income, errors = 'coerce')
times.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
0,1,Harvard University,United States of America,99.7,72.4,98.7,98.8,34.5,96.1,20152.0,8.9,25%,NaN,2011
1,2,California Institute of Technology,United States of America,97.7,54.6,98.0,99.9,83.7,96.0,2243.0,6.9,27%,33 : 67,2011
2,3,Massachusetts Institute of Technology,United States of America,97.8,82.3,91.4,99.9,87.5,95.6,11074.0,9.0,33%,37 : 63,2011
3,4,Stanford University,United States of America,98.3,29.5,98.1,99.2,64.3,94.3,15596.0,7.8,22%,42 : 58,2011
4,5,Princeton University,United States of America,90.9,70.3,95.4,99.9,NaN,94.2,7929.0,8.4,27%,45 : 55,2011


In [7]:
shangai.head()

,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year
0,1,Harvard University,1,100.0,100.0,100.0,100.0,100.0,100.0,72.4,2005
1,2,University of Cambridge,1,73.6,99.8,93.4,53.3,56.6,70.9,66.9,2005
2,3,Stanford University,2,73.4,41.1,72.2,88.5,70.9,72.3,65.0,2005
3,4,"University of California, Berkeley",3,72.8,71.8,76.0,69.4,73.9,72.2,52.7,2005
4,5,Massachusetts Institute of Technology (MIT),4,70.1,74.0,80.6,66.7,65.8,64.3,53.0,2005


In [8]:
cwur.head()

,world_rank,institution,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year
0,1,Harvard University,USA,1,7,9,1,1,1,1,NaN,5,100.00,2012
1,2,Massachusetts Institute of Technology,USA,2,9,17,3,12,4,4,NaN,1,91.67,2012
2,3,Stanford University,USA,3,17,11,5,4,2,2,NaN,15,89.50,2012
3,4,University of Cambridge,United Kingdom,1,10,24,4,16,16,11,NaN,50,86.17,2012
4,5,California Institute of Technology,USA,4,2,29,7,37,22,22,NaN,18,85.21,2012


### 1. For each university, extract from the times dataset the most recent and the least recent data, obtaining two separate dataframes.


L'obiettivo è ottenere due dataframe separati contenenti le informazioni più e meno recenti per ogni università contenuti nel dataframe `times`. <br/> Tale risultato può essere ottenuto sfruttando i metodi di Pandas `idxmax` e `idxmin`, a seguito di un raggruppamento rispetto al nome delle università. <br/>
Questi metodi consentono di ottenere le posizioni dei record relativi rispettivamente al valore massimo e minimo. Una volta ottenuti questi indici è sufficiente sfruttare il metodo `loc` per estrarre l'informazione corrispondente. 

In [9]:
new_times = times.loc[times.groupby('university_name').year.idxmax()]
new_times.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,NaN,-,35569.0,17.0,1%,-,2016
2003,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,-,17422.0,15.9,15%,48 : 52,2016
2056,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,-,16099.0,24.2,17%,32 : 68,2016
1908,=106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,23895.0,13.6,14%,54 : 46,2016
2105,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,9252.0,19.2,18%,48 : 52,2016


In [10]:
old_times = times.loc[times.groupby('university_name').year.idxmin()]
old_times.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,NaN,-,35569.0,17.0,1%,-,2016
501,301-350,Aalborg University,Denmark,19.0,75.3,20.0,27.1,36.4,-,17422.0,15.9,15%,48 : 52,2012
502,301-350,Aalto University,Finland,26.2,49.0,22.2,37.5,61.9,-,16099.0,24.2,17%,32 : 68,2012
166,167,Aarhus University,Denmark,38.1,33.4,55.6,57.3,61.5,49.9,23895.0,13.6,14%,54 : 46,2011
476,276-300,Aberystwyth University,United Kingdom,19.8,63.8,15.5,56.6,35.5,-,9252.0,19.2,18%,48 : 52,2012


### 2. For each university, compute the improvement in income between the least recent and the most recent data points.


L'idea è di performare una join dei dataset costruiti nel punto precedente sul nome dell'università e di valutare da questa unione l'eventuale differenza nei livelli di reddito. 

In [11]:
merged = pd.merge(new_times, old_times, on = 'university_name', suffixes = ('_new', '_old'))
merged.head()

,world_rank_new,university_name,country_new,teaching_new,international_new,research_new,citations_new,income_new,total_score_new,num_students_new,...,international_old,research_old,citations_old,income_old,total_score_old,num_students_old,student_staff_ratio_old,international_students_old,female_male_ratio_old,year_old
0,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,NaN,-,35569.0,...,17.9,3.7,35.7,NaN,-,35569.0,17.0,1%,-,2016
1,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,-,17422.0,...,75.3,20.0,27.1,36.4,-,17422.0,15.9,15%,48 : 52,2012
2,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,-,16099.0,...,49.0,22.2,37.5,61.9,-,16099.0,24.2,17%,32 : 68,2012
3,=106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,23895.0,...,33.4,55.6,57.3,61.5,49.9,23895.0,13.6,14%,54 : 46,2011
4,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,9252.0,...,63.8,15.5,56.6,35.5,-,9252.0,19.2,18%,48 : 52,2012


E' importante individuare eventuali valori mancanti nei campi di interesse prima di effettuare il calcolo della variazione del reddito. Questa operazione si può effettuare aggiungendo il comando `all` al metodo `notna()`, il quale restituisce `False` se anche un solo valore della serie è nullo.

In [12]:
merged.income_new.notna().all()

False

In [13]:
merged.income_old.notna().all()

False

In entrambi i dataframe sono presenti record con valori nulli alla voce `income`. Al fine di risolvere questa problematica si è optato per la rimozione dei record con valori mancanti.

In [14]:
merged.dropna(subset = ['income_new', 'income_old'], inplace = True)

In [15]:
merged['improvement'] = merged.income_new - merged.income_old 

In [16]:
merged[['university_name', 'improvement']].head()

,university_name,improvement
1,Aalborg University,7.3
2,Aalto University,-0.3
3,Aarhus University,6.8
4,Aberystwyth University,-4.2
5,Adam Mickiewicz University,0.0


### 3. Find the university with the largest increase computed in the previous point.


Per individuare l'università con la maggiore crescita si può sfruttare nuovamente il metodo `idxmax` con riferimento alla colonna `improvement`. <br/> E' doveroso però effettuare un controllo per avere evidenza di università che presentino il medesimo miglioramento. Per questa operazione si effettua un ordinamento in base al miglioramento e si controllano eventuali valori massimi ripetuti.

In [17]:
merged.improvement.sort_values(ascending = False).head()

428    67.8
278    59.9
230    59.6
205    59.5
276    58.3
Name: improvement, dtype: float64

L'università *TU Dresden* risulta essere quella che ha registrato un maggiore incremento nel livello di reddito nel periodo di analisi (pari a 67.8).

In [18]:
merged.loc[merged['improvement'].idxmax()]

world_rank_new                      =158
university_name               TU Dresden
country_new                      Germany
teaching_new                        41.4
international_new                   47.7
research_new                        45.8
citations_new                       66.1
income_new                          99.7
total_score_new                     52.1
num_students_new                   35487
student_staff_ratio_new             37.4
international_students_new           12%
female_male_ratio_new            42 : 58
year_new                            2016
world_rank_old                   251-275
country_old                      Germany
teaching_old                        27.3
international_old                   49.2
research_old                        13.8
citations_old                       57.4
income_old                          31.9
total_score_old                        -
num_students_old                   35487
student_staff_ratio_old             37.4
international_st

### 4. For each ranking, consider only the most recent data point. For each university, compute the maximum difference between the rankings (e.g. for Aarhus University the value is 122-73=49). Notice that some rankings are expressed as a range.

Nei ranking sono presenti alcuni problemi di eterogeneità relativi al nome delle università. Ad esempio, nel ranking `shangai` l'università *Massachusetts Institute of Technology* viene indentificata anche con la nota sigla *MIT* racchiusa tra parentesi. Per questo motivo, è necessario normalizzare preventivamente i nomi delle università nei diversi ranking. <br/>
Le operazioni di seguito hanno quindi l'obiettivo di rendere tutte le stringhe minuscole e ad eliminare eventuali parentesi.

Infine, nel dataframe `times`, alcuni record hanno un valore della posizione preceduta da un segno `=` che va necessariamente eliminato per poter trattare tutte le posizioni come valori numerici.

Per svolgere queste operazioni sfrutteremo sia le espressioni regolari fornite dalla libreria `re`, sia i metodi associati alle stringhe.

In [19]:
times.world_rank.tail()

2598    601-800
2599    601-800
2600    601-800
2601    601-800
2602    601-800
Name: world_rank, dtype: object

Per prima cosa si procedere alla compilazione dell'espressione regolare che verrà usata per gestire gli intervalli di valori.

In [20]:
src = re.compile('(?P<first>\d+)\s*-*\s*(?P<second>\d*)')

###### `times`

In [21]:
times.university_name = times.university_name.str.lower()
times.university_name.head()

0                       harvard university
1       california institute of technology
2    massachusetts institute of technology
3                      stanford university
4                     princeton university
Name: university_name, dtype: object

In [22]:
times.university_name = times.university_name.str.replace('\(\w+\)$', '', regex = True)
times.university_name.head()

0                       harvard university
1       california institute of technology
2    massachusetts institute of technology
3                      stanford university
4                     princeton university
Name: university_name, dtype: object

In [23]:
times.world_rank = times.world_rank.str.strip('=')

###### `shangai`

In [24]:
shangai.university_name = shangai.university_name.str.lower()
shangai.university_name.head()

0                             harvard university
1                        university of cambridge
2                            stanford university
3             university of california, berkeley
4    massachusetts institute of technology (mit)
Name: university_name, dtype: object

In [25]:
shangai.university_name = shangai.university_name.str.replace('\(\w+\)$', '', regex = True)
shangai.university_name.head()

0                        harvard university
1                   university of cambridge
2                       stanford university
3        university of california, berkeley
4    massachusetts institute of technology 
Name: university_name, dtype: object

###### `cwur`

In [26]:
cwur.institution = cwur.institution.str.lower()
cwur.institution.head()

0                       harvard university
1    massachusetts institute of technology
2                      stanford university
3                  university of cambridge
4       california institute of technology
Name: institution, dtype: object

In [27]:
cwur.institution = cwur.institution.str.replace('\(\w+\)$', '', regex = True)
cwur.institution.head()

0                       harvard university
1    massachusetts institute of technology
2                      stanford university
3                  university of cambridge
4       california institute of technology
Name: institution, dtype: object

A questo punto è possibile proseguire nelle operazioni di analisi. Per prima cosa individuiamo l'anno più recente contenuto nei diversi ranking ed andiamo ad integrarlo ai precedenti dataset considerando come chiave il nome dell'università.

In [28]:
times_years = set(times.year)
times_years

{2011, 2012, 2013, 2014, 2015, 2016}

In [29]:
shangai_years = set(shangai.year)
shangai_years

{2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015}

In [30]:
cwur_years = set(cwur.year)
cwur_years

{2012, 2013, 2014, 2015}

In [31]:
rank = pd.merge(times[times.year == max(times_years)].add_suffix('_times'), shangai[shangai.year == max(shangai_years)].add_suffix('_shangai'), left_on = 'university_name_times', right_on = 'university_name_shangai')
rank = rank.merge(cwur[cwur.year == max(cwur_years)].add_suffix('_cwur'), right_on = 'institution_cwur', left_on = 'university_name_times')
rank.rename({'university_name_times':'university_name'}, axis = 'columns', inplace = True)

E' bene osservare che nei dataset `shangai` e `times` il ranking è espresso non soltanto con valori puntuali ma anche per mezzo di intervalli. Quest'ultimi verranno quindi elaborati in modo tale da ottenere il valore centrale degli stessi, così da poter verificare infine la massima differenza tra le varie posizioni.

In [32]:
for i in range(len(rank)):
    fnd = src.search(rank.loc[i,'world_rank_shangai'])
    if fnd.group('second'): 
        rank.loc[i, 'world_rank_shangai_2'] = (int(fnd.group('first')) + int(fnd.group('second')))/2
    else:
         rank.loc[i, 'world_rank_shangai_2'] = int(fnd.group('first'))

In [33]:
for i in range(len(rank)):
    fnd = src.search(rank.loc[i,'world_rank_times'])
    if fnd.group('second'): 
        rank.loc[i, 'world_rank_times_2'] = (int(fnd.group('first')) + int(fnd.group('second')))/2
    else:
         rank.loc[i, 'world_rank_times_2'] = int(fnd.group('first'))

In [34]:
rank.columns

Index(['world_rank_times', 'university_name', 'country_times',
       'teaching_times', 'international_times', 'research_times',
       'citations_times', 'income_times', 'total_score_times',
       'num_students_times', 'student_staff_ratio_times',
       'international_students_times', 'female_male_ratio_times', 'year_times',
       'world_rank_shangai', 'university_name_shangai',
       'national_rank_shangai', 'total_score_shangai', 'alumni_shangai',
       'award_shangai', 'hici_shangai', 'ns_shangai', 'pub_shangai',
       'pcp_shangai', 'year_shangai', 'world_rank_cwur', 'institution_cwur',
       'country_cwur', 'national_rank_cwur', 'quality_of_education_cwur',
       'alumni_employment_cwur', 'quality_of_faculty_cwur',
       'publications_cwur', 'influence_cwur', 'citations_cwur',
       'broad_impact_cwur', 'patents_cwur', 'score_cwur', 'year_cwur',
       'world_rank_shangai_2', 'world_rank_times_2'],
      dtype='object')

Il dataframe così costruito presenta un'ampia complessità dovuta alla grande mole di colonne. Ma solo tre di queste sono di nostro interesse: `world_rank_times_2`,`world_rank_shangai_2`, `world_rank_cwur`. <br/> Si noti che per distinguere il posizionamento espresso come intervallo da quello puntuale, per i primi due ranking è stato aggiunto il suffisso `_2`.

In [35]:
rank[['world_rank_times_2','world_rank_shangai_2', 'world_rank_cwur']].head()

,world_rank_times_2,world_rank_shangai_2,world_rank_cwur
0,1.0,7.0,12
1,2.0,10.0,5
2,3.0,2.0,2
3,4.0,5.0,4
4,6.0,1.0,1


Con le informazioni contenute del dataframe precedente è possibile calcolare le differenze di classifica e selezionare così quella massima tra i diversi ranking.

In [36]:
for i in range(len(rank)):
    rank.loc[i,'max_diff'] = max(abs(rank.loc[i,'world_rank_times_2'] - rank.loc[i,'world_rank_shangai_2']), abs(rank.loc[i,'world_rank_times_2'] - rank.loc[i,'world_rank_cwur']), abs(rank.loc[i,'world_rank_shangai_2'] - rank.loc[i,'world_rank_cwur']))

In [37]:
rank.sort_values('max_diff', ascending = False)[['university_name','world_rank_times_2', 'world_rank_shangai_2','world_rank_cwur', 'max_diff']].head()

,university_name,world_rank_times_2,world_rank_shangai_2,world_rank_cwur,max_diff
162,king abdulaziz university,275.5,175.5,995,819.5
297,university of tehran,700.5,250.5,768,517.5
265,keio university,550.5,350.5,34,516.5
293,okayama university,700.5,450.5,245,455.5
289,jilin university,700.5,250.5,391,450.0


Per validare il risultato dell'operazione effettuiamo un confronto col valore dell'università `Aarhus university` (max_diff = **49**).

In [38]:
rank[rank.university_name == 'aarhus university'].max_diff

65    49.0
Name: max_diff, dtype: float64

### 5. Consider only the most recent data point of the times dataset. Compute the number of male and female students for each country.


Per prima cosa estraiamo i dati più recenti dal dataframe `times`. Si effettua quindi una copia del dataframe in modo da operare in maggiore libertà sul subset. In questo caso non è un problema dato che la dimensione del dataframe originale non è tale da rendere la copia svantaggiosa.

In [39]:
recent_times = times[times.year == max(times_years)].copy()

In [40]:
recent_times.female_male_ratio.isnull().any()

True

Essendo presente un valore nullo si opta anche in questo caso per una rimozione di tale valore. Si effettua anche un controllo sui rapporti definiti come stringhe per verificare siano tutti validi.

In [41]:
recent_times.dropna(subset = ['female_male_ratio'], inplace = True)

In [42]:
recent_times = recent_times[recent_times['female_male_ratio'].str.contains('\d+\s*:\s*\d+')]

In [43]:
recent_times.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
1803,1,california institute of technology,United States of America,95.6,64.0,97.6,99.8,97.8,95.2,2243.0,6.9,27%,33 : 67,2016
1804,2,university of oxford,United Kingdom,86.5,94.4,98.9,98.8,73.1,94.2,19919.0,11.6,34%,46 : 54,2016
1805,3,stanford university,United States of America,92.5,76.3,96.2,99.9,63.3,93.9,15596.0,7.8,22%,42 : 58,2016
1806,4,university of cambridge,United Kingdom,88.2,91.5,96.7,97.0,55.0,92.8,18812.0,11.8,34%,46 : 54,2016
1807,5,massachusetts institute of technology,United States of America,89.4,84.0,88.6,99.7,95.4,92.0,11074.0,9.0,33%,37 : 63,2016


In [44]:
recent_times['num_students'].dtype

dtype('float64')

Si effettua ora una conversione numerica del rapporto $\frac{femmina}{maschio}$ a partire dalla colonna di stringhe e successivamente si calcolano il numero di maschi e femmine sfruttando il numero di studenti totale.

In [45]:
recent_times[['female', 'male']] = round((recent_times['female_male_ratio'].str.split(' : ', expand = True).astype('int')/100).mul(recent_times['num_students'], axis = 0))

In [46]:
recent_times[['female', 'male']].sort_values('male').head()

,female,male
2227,17625.0,0.0
1915,129.0,333.0
2036,1401.0,418.0
1983,269.0,500.0
2037,1317.0,538.0


A questo punto è sufficiente raggruppare in base alla nazione di appartenenza e sommare il numero di studenti maschili e femminili per ognuna di queste, ottenendo quindi il risultato di interesse.

In [47]:
recent_times[['country','female', 'male']].groupby('country').sum().head()

,female,male
country,,
Argentina,67191.0,41182.0
Australia,391736.0,321640.0
Austria,68364.0,66113.0
Bangladesh,21323.0,41393.0
Belarus,20219.0,9084.0


### 6. Find the universities where the ratio between female and male is below the average ratio (computed over all universities).

In [48]:
recent_times[recent_times.male == 0]

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,female,male
2227,401-500,ewha womans university,South Korea,23.9,34.3,18.3,50.4,57.6,-,17625.0,14.4,6%,100 : 0,2016,17625.0,0.0


In [49]:
recent_times[recent_times['female_male_ratio'].isna()]

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,female,male


Da una prima analisi, non risultano essere presenti valori nulli per il rapporto che si vuole analizzare. Tuttavia c'è un problema legato all'università femminile `ewha womans university`, la quale fornisce un rapporto $\frac{female}{male}$ pari a $\infty$. Di conseguenza, considerare questa università nel calcolo della media porterebbe questa a divergere. <br/>
Per questo motivo la `ewha womans university` viene esclusa e considerata a priori superiore alla media.

In [50]:
recent_times_under = recent_times[recent_times['male'] != 0].copy()

In [51]:
recent_times_under['proper_ratio'] = recent_times['female'] / recent_times['male']
average_ratio = recent_times_under['proper_ratio'].mean() 
recent_times_under = recent_times_under[recent_times_under['proper_ratio'] < average_ratio]

In [52]:
average_ratio

1.0825701851588103

In [53]:
recent_times_under[['world_rank', 'university_name', 'proper_ratio']].head()

,world_rank,university_name,proper_ratio
1803,1,california institute of technology,0.492349
1804,2,university of oxford,0.851897
1805,3,stanford university,0.724077
1806,4,university of cambridge,0.851939
1807,5,massachusetts institute of technology,0.587215


### 7. For each country compute the fraction of students that are in one of the universities computed in the previous point (that is, the denominator of the ratio is the total number of students over all universities in the country).

Per rispondere al quesito vengono create due serie: una contenente il numero di studenti per nazione ed una per università. Si uniscono le due sulla base della nazione in modo da poter calcolare infine i rapporti, facendo leva sulle funzionalità di Pandas.

In [54]:
student_university = recent_times_under.groupby(['university_name', 'country'])['num_students'].sum()
student_university.head()

university_name         country       
aalborg university      Denmark            17422.0
aalto university        Finland            16099.0
aberystwyth university  United Kingdom      9252.0
ajou university         South Korea        12706.0
alexandria university   Egypt             127431.0
Name: num_students, dtype: float64

In [55]:
student_country = recent_times.groupby('country')['num_students'].sum()

In [56]:
student_merged = pd.merge(student_country, student_university, right_on = 'country', left_index = True, suffixes = ('_country', '_university') )
student_merged.reset_index(inplace = True)
student_merged.head()

,university_name,country,num_students_country,num_students_university
0,rmit university,Australia,713376.0,29501.0
1,swinburne university of technology,Australia,713376.0,20314.0
2,university of adelaide,Australia,713376.0,20771.0
3,university of new south wales,Australia,713376.0,38309.0
4,university of western australia,Australia,713376.0,20851.0


In [57]:
student_merged['ratio'] = student_merged['num_students_university']/student_merged['num_students_country']
student_merged.head()

,university_name,country,num_students_country,num_students_university,ratio
0,rmit university,Australia,713376.0,29501.0,0.041354
1,swinburne university of technology,Australia,713376.0,20314.0,0.028476
2,university of adelaide,Australia,713376.0,20771.0,0.029116
3,university of new south wales,Australia,713376.0,38309.0,0.053701
4,university of western australia,Australia,713376.0,20851.0,0.029229


### 8. Read the file educational_attainment_supplementary_data.csv, discarding any row without country_name or series_name.

Per rispondere al quesito, è stato utilizzato il metodo `dropna` con flag `subset` per eliminare le righe con `null` nelle due colonne indicate. Si è verificata quindi la riduzione nel numero di righe con il metodo `shape`.

In [58]:
edu_att_supp_data = pd.read_csv('https://github.com/DanieleRizz0/CSProject/blob/main/educational_attainment_supplementary_data.csv?raw=true')

In [59]:
edu_att_supp_data.shape

(79055, 29)

In [60]:
edu_att_supp_data.dropna(subset = ['country_name', 'series_name'], inplace = True)
edu_att_supp_data.head()

,country_name,series_name,1985,1986,1987,1990,1991,1992,1993,1995,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2015
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.33,NaN,NaN,0.44,NaN,NaN,NaN,0.57,...,0.86,NaN,NaN,NaN,NaN,1.27,NaN,NaN,NaN,NaN
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1.03,NaN,NaN,1.26,NaN,NaN,NaN,1.54,...,2.18,NaN,NaN,NaN,NaN,2.64,NaN,NaN,NaN,NaN
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.83,NaN,NaN,0.95,NaN,NaN,NaN,1.26,...,1.01,NaN,NaN,NaN,NaN,2.45,NaN,NaN,NaN,NaN
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",2.34,NaN,NaN,2.22,NaN,NaN,NaN,2.37,...,2.26,NaN,NaN,NaN,NaN,3.55,NaN,NaN,NaN,NaN
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.54,NaN,NaN,0.92,NaN,NaN,NaN,0.94,...,2.00,NaN,NaN,NaN,NaN,1.29,NaN,NaN,NaN,NaN


In [61]:
edu_att_supp_data.shape

(79050, 29)

### 9. From attainment build a dataframe with the same data, but with 4 columns: country_name, series_name, year, value.

Per svolgere quanto richiesto, è stato utilizzato il metodo di Pandas `stack`, che consente di trasporre le colonne del dataframe rappresentative degli anni, passando così da un dataset di tipo key figure-based ad uno account-based. Per non perdere i valori nulli nel calcolo, viene inoltre settato il flag `dropna` del metodo su `False`.

In [62]:
edu_att_supp_data.head()

,country_name,series_name,1985,1986,1987,1990,1991,1992,1993,1995,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2015
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.33,NaN,NaN,0.44,NaN,NaN,NaN,0.57,...,0.86,NaN,NaN,NaN,NaN,1.27,NaN,NaN,NaN,NaN
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1.03,NaN,NaN,1.26,NaN,NaN,NaN,1.54,...,2.18,NaN,NaN,NaN,NaN,2.64,NaN,NaN,NaN,NaN
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.83,NaN,NaN,0.95,NaN,NaN,NaN,1.26,...,1.01,NaN,NaN,NaN,NaN,2.45,NaN,NaN,NaN,NaN
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",2.34,NaN,NaN,2.22,NaN,NaN,NaN,2.37,...,2.26,NaN,NaN,NaN,NaN,3.55,NaN,NaN,NaN,NaN
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.54,NaN,NaN,0.92,NaN,NaN,NaN,0.94,...,2.00,NaN,NaN,NaN,NaN,1.29,NaN,NaN,NaN,NaN


In [63]:
four_columns_att = edu_att_supp_data.set_index(['country_name', 'series_name'])
four_columns_att = four_columns_att.stack(dropna = False).reset_index()
four_columns_att.rename(columns = {'level_2': 'year', 0: 'value'}, inplace = True)
four_columns_att.head()

,country_name,series_name,year,value
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.33
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1986,NaN
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1987,NaN
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1990,0.44
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1991,NaN


### 10. For each university, find the number of rankings in which they appear (it suffices to appear in one year for each ranking).

Per questa rispondere al quesito si è scelto di sfruttare le operazioni insiemistiche. Sono state quindi inserite le università presenti nei diversi ranking in altrettanti set. E' stato effettuato quindi un conteggio dopo avere unito i suddetti set, al fine di ottenere il numero di volte in cui un'università si trova all'interno dell'insieme complessivo.

In [64]:
uni_times = set(times['university_name'])
uni_shangai = set(shangai['university_name'])
uni_cwur = set(cwur['institution'])

In [65]:
universities = pd.DataFrame(list(uni_times) + list(uni_shangai) + list(uni_cwur), columns = ['university'])

In [66]:
universities.groupby('university').size().head()

university
aalborg university            3
aalto university              3
aarhus university             3
aberystwyth university        2
adam mickiewicz university    1
dtype: int64

### 11. In the times ranking, compute the number of times each university appears.

Per questo punto si raggruppa il dataframe in base all'università utilizzando la funzione `groupby` di Pandas e si effettua un conteggio.

In [67]:
times.groupby('university_name').size().head()

university_name
aalborg university            5
aalto university              5
aarhus university             6
aberystwyth university        5
adam mickiewicz university    1
dtype: int64

### 12. Find the universities that appear at most twice in the times ranking.


Si definisce una maschera booleana tale da ritornare `True` se le università sono presenti al più due volte nel ranking `times`.
Infine, la maschera viene sfruttata per estrarre le università corrispondenti.

In [68]:
tf = times.groupby('university_name').size() <= 2
tf.head()

university_name
aalborg university            False
aalto university              False
aarhus university             False
aberystwyth university        False
adam mickiewicz university     True
dtype: bool

In [69]:
list(set(tf[tf == True].index))[:10]

['university of parma',
 'xiamen university',
 'yeungnam university',
 'university of nantes',
 'american university',
 'adam mickiewicz university',
 'university of st gallen',
 'medical college of wisconsin',
 'university of genoa',
 'university of rovira i virgili']

### 13. The universities that, in any year, have the same position in all three rankings (they must have the same position in a year).

E' stato costruito un dataframe, denominato `full_rank`, operando una join dei dataset `times`, `shangai` e `cwur`, considerando come predicato non solo il nome dell'università ma anche l'anno di riferimento del ranking. 
Fatto ciò, viene cercata l'osservazione che ha fatto registrare la stessa posizione nei diversi ranking e, per costruzione del dataframe `full_rank`, anche nello stesso anno.

In [70]:
cwur.rename(columns = {'institution' : 'university_name'}, inplace = True)
cwur.head()

,world_rank,university_name,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year
0,1,harvard university,USA,1,7,9,1,1,1,1,NaN,5,100.00,2012
1,2,massachusetts institute of technology,USA,2,9,17,3,12,4,4,NaN,1,91.67,2012
2,3,stanford university,USA,3,17,11,5,4,2,2,NaN,15,89.50,2012
3,4,university of cambridge,United Kingdom,1,10,24,4,16,16,11,NaN,50,86.17,2012
4,5,california institute of technology,USA,4,2,29,7,37,22,22,NaN,18,85.21,2012


In [71]:
full_rank = times.merge(cwur, on = ['university_name', 'year'], how = 'inner' )
full_rank = full_rank.merge(shangai, on = ['university_name', 'year'], how = 'inner' )
full_rank.rename(columns = {'world_rank_x':'times_rank', 'world_rank_y':'cwur_rank', 'world_rank':'shangai_rank'}, inplace=True )

In [72]:
full_rank['cwur_rank'] = full_rank['cwur_rank'].astype(str)

In [73]:
tmp_merged = full_rank[(full_rank['times_rank'] == full_rank['cwur_rank']) & (full_rank['cwur_rank'] == full_rank['shangai_rank'])]

In [74]:
tmp_merged[['university_name','year','times_rank','cwur_rank','shangai_rank']]

,university_name,year,times_rank,cwur_rank,shangai_rank
60,stanford university,2013,2,2,2


In [75]:
end = time.time()

print("Il processo complessivo è stato ultimato in {} secondi.".format(int((end-start))))

Il processo complessivo è stato ultimato in 19 secondi.
